In [11]:
#6
#a Load the Data

import pandas as pd

# Load the data from the CSV file
data = pd.read_csv("D:/My Drive/Atazonomics World/BSE_2023-2024/T1/Computing for Data Science/Psets/Pset3/sample_diabetes_mellitus_data.csv")

# Display the first few rows of the dat
data.head()


,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,0,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,...,0,14.1,0,0,0,0,0,0,0,1
1,1,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,...,1,12.7,0,0,0,0,0,0,0,1
2,2,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,...,0,NaN,0,0,0,0,0,0,0,0
3,3,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,...,1,8.0,0,0,0,0,0,0,0,0
4,4,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,0,NaN,0,0,0,0,0,0,0,0


In [12]:
#b. Split the Data:Training and Testing

from sklearn.model_selection import train_test_split

# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

train_data.shape, test_data.shape


((8000, 53), (2000, 53))

In [13]:
#Preprocess the Data

#Training Data 
# c. Remove rows with NaN values in specified columns
train_data = train_data.dropna(subset=['age', 'gender', 'ethnicity'])

# d. Fill NaN values with the mean of the column for specified columns
for col in ['height', 'weight']:
    train_data[col].fillna(train_data[col].mean(), inplace=True)

# e. Generate dummies for the 'ethnicity' column
train_data = pd.get_dummies(train_data, columns=['ethnicity'], drop_first=True)

# f. Create a binary variable for the 'gender' column
train_data['gender'] = train_data['gender'].apply(lambda x: 1 if x == 'M' else 0)

# Display a subset of the processed data
train_data[['age', 'gender', 'height', 'weight']].head()  

#Test Data 
# c. Remove rows with NaN values in specified columns
test_data = test_data.dropna(subset=['age', 'gender', 'ethnicity'])

# d. Fill NaN values with the mean of the column for specified columns
for col in ['height', 'weight']:
    test_data[col].fillna(test_data[col].mean(), inplace=True)

# e. Generate dummies for the 'ethnicity' column
test_data = pd.get_dummies(test_data, columns=['ethnicity'], drop_first=True)

# f. Create a binary variable for the 'gender' column
test_data['gender'] = test_data['gender'].apply(lambda x: 1 if x == 'M' else 0)

# Display a subset of the processed data
test_data[['age', 'gender', 'height', 'weight']].head()  


,age,gender,height,weight
6252,66.0,1,180.3,113.3
1731,83.0,0,154.9,67.7
4742,64.0,0,170.2,62.5
4521,21.0,0,162.0,90.7
6340,66.0,0,137.2,76.0


In [14]:
#g. Train a model using the specified features and target.
#h. Predict the targets for both the train and test sets and add the predictions as a new column.
#F We use RandomForestClassifier train the model. 

#import Package 

from sklearn.ensemble import RandomForestClassifier

# Define the features and target
features = ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure', 
            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']
target = 'diabetes_mellitus'

# Initialize the model
clf = RandomForestClassifier(random_state=42)

# Train the model
clf.fit(train_data[features], train_data[target])

# h. Predict the targets for the train and test sets
train_data['predictions'] = clf.predict_proba(train_data[features])[:, 1]
test_data['predictions'] = clf.predict_proba(test_data[features])[:, 1]
 # Display actual target vs predicted probabilities
train_data[['diabetes_mellitus', 'predictions']].head() 


,diabetes_mellitus,predictions
9254,1,0.820000
1561,1,0.482524
1670,0,0.100000
6087,0,0.223333
6669,0,0.060000


In [15]:
#i. Compute the train and test roc_auc metric using roc_auc_score

from sklearn.metrics import roc_auc_score

# Compute ROC AUC for train and test sets
train_roc_auc = roc_auc_score(train_data[target], train_data['predictions'])
test_roc_auc = roc_auc_score(test_data[target], test_data['predictions'])

train_roc_auc, test_roc_auc

# Training set ROC AUC: 0.99780.9978
 #Testing set ROC AUC: 0.59550.5955

# Model performs exceptionally well on the training data, but only moderate on the test data
# Suggesting that the model might be overfitting the training data.



(0.9978081247085744, 0.595543242194274)